In [1]:
import boto3
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder

In [2]:
client = boto3.client('s3')

In [3]:
path_train = "s3://quri-sage-maker-bucket/new_FE_train.feather770"

In [4]:
train = pd.read_feather(path_train)

In [5]:
train.shape

(458913, 772)

In [6]:
train.sample()

,customer_ID,B_30_nunique,B_38_last,B_38_nunique,D_114_last,D_117_last,D_120_last,D_120_nunique,D_66_last,D_66_nunique,D_64_last,D_63_last,D_63_nunique,D_126_nunique,S_2_min,S_27_delta,S_7_delta,S_5_delta,R_5_delta,S_11_delta,D_142_delta,D_43_delta,D_136_delta,S_22_delta,D_110_delta,R_28_delta,B_6_delta,D_102_delta,D_130_delta,D_46_delta,D_137_delta,B_1_delta,D_96_delta,R_14_delta,D_72_delta,S_12_delta,S_13_delta,D_88_delta,B_24_delta,S_25_delta,D_44_delta,B_9_delta,B_17_delta,B_42_delta,D_53_delta,D_58_delta,R_11_delta,D_138_delta,B_4_delta,B_3_delta,R_16_delta,SDist_delta,D_48_delta,D_84_delta,R_7_delta,B_2_delta,D_39_delta,D_41_delta,D_82_delta,D_49_delta,B_16_delta,LT,D_127_quantile,B_22_quantile,D_76_quantile,R_14_quantile,D_121_quantile,S_12_quantile,B_2_quantile,D_56_quantile,D_50_quantile,D_47_quantile,SDist_quantile,R_5_quantile,R_4_quantile,D_87_quantile,D_128_quantile,D_139_quantile,D_45_quantile,D_48_quantile,D_59_quantile,S_18_quantile,D_65_quantile,S_15_quantile,D_141_quantile,S_13_quantile,D_143_quantile,B_13_quantile,D_119_quantile,S_26_quantile,D_43_quantile,D_134_quantile,D_51_quantile,R_7_quantile,B_17_quantile,D_94_quantile,B_3_quantile,D_122_quantile,S_3_quantile,D_91_quantile,D_72_quantile,P_4_quantile,B_40_quantile,R_8_quantile,S_24_quantile,B_10_quantile,D_60_quantile,D_71_quantile,B_42_quantile,D_111_quantile,D_110_quantile,B_9_quantile,D_61_quantile,P_2_quantile,R_26_quantile,D_96_quantile,D_70_quantile,B_14_quantile,D_75_quantile,D_92_quantile,D_74_quantile,R_25_quantile,D_107_quantile,D_42_quantile,R_9_quantile,D_62_quantile,R_27_quantile,D_93_quantile,D_125_quantile,B_32_quantile,B_5_quantile,D_105_quantile,R_24_quantile,D_44_quantile,B_20_mean,R_23_mean,D_59_mean,D_140_mean,D_75_mean,D_106_mean,R_8_mean,D_130_mean,R_26_mean,D_74_mean,R_15_mean,D_52_mean,D_54_mean,R_9_mean,D_121_mean,D_113_mean,S_26_mean,B_14_mean,D_50_mean,S_12_mean,S_15_mean,D_93_mean,B_32_mean,D_73_mean,D_111_mean,D_108_mean,D_77_mean,R_21_mean,SDist_mean,B_9_mean,B_17_mean,D_141_mean,S_5_mean,D_96_mean,B_21_mean,R_1_mean,D_48_mean,D_61_mean,R_5_mean,S_16_mean,D_92_mean,D_91_mean,D_45_mean,B_40_mean,D_110_mean,B_18_mean,D_82_mean,R_27_mean,S_3_mean,D_71_mean,B_42_mean,R_16_mean,P_2_mean,D_44_mean,D_145_mean,D_122_mean,B_4_mean,D_55_mean,B_15_mean,D_104_mean,D_62_mean,S_6_mean,B_5_mean,D_94_mean,D_72_mean,S_11_mean,D_127_mean,D_142_mean,D_143_mean,D_76_mean,B_41_mean,S_18_mean,D_60_mean,R_3_mean,R_11_mean,B_7_mean,D_118_mean,D_51_mean,R_25_mean,B_2_mean,D_42_mean,B_22_mean,D_47_mean,D_43_mean,B_12_mean,D_83_last,B_9_last,R_5_last,R_11_last,R_16_last,D_50_last,S_25_last,D_111_last,B_4_last,D_76_last,R_2_last,S_24_last,S_3_last,P_2_last,D_119_last,B_15_last,R_9_last,B_24_last,D_140_last,D_52_last,SDist_last,D_79_last,B_23_last,D_127_last,D_49_last,B_1_last,D_59_last,S_26_last,R_20_last,B_18_last,S_23_last,...,S_5_std,D_51_std,D_123_std,B_4_std,R_14_std,B_19_std,S_13_std,B_2_std,B_17_std,D_41_std,B_14_std,B_41_std,B_39_std,D_47_std,S_20_std,R_3_std,D_87_std,D_84_std,S_15_std,B_12_std,D_137_std,B_23_std,SDist_std,D_45_std,S_18_std,R_20_std,P_3_std,R_7_std,D_61_std,B_16_std,S_23_std,R_27_min,D_46_min,S_7_min,D_118_min,R_12_min,B_11_min,S_5_min,B_5_min,D_76_min,D_47_min,D_119_min,R_2_min,D_70_min,D_48_min,B_16_min,R_28_min,D_53_min,D_111_min,D_51_min,B_13_min,P_4_min,D_56_min,S_19_min,B_6_min,D_135_min,R_4_min,D_52_min,S_23_min,D_141_min,R_9_min,D_61_min,S_15_min,D_109_min,S_12_min,B_39_min,D_62_min,D_132_min,D_112_min,B_20_min,R_6_min,D_110_min,D_104_min,B_32_min,D_91_min,D_140_min,B_40_min,B_9_min,R_8_min,D_92_min,S_3_min,B_18_min,D_42_min,D_93_min,B_31_min,D_125_min,SDist_min,B_42_min,S_8_min,R_5_min,R_11_min,B_33_min,D_83_min,R_23_min,D_127_min,S_25_min,D_121_min,D_94_min,D_65_min,B_8_min,S_11_min,D_134_min,B_17_min,D_39_min,S_6_min,D_45_min,D_122_min,D_59_min,D_84_min,D_42_last_round2,B_17_last_round2,B_3_last_round2,D_62_last_round2,B_18_last_round2,R_12_last_round2,D_133_last_round2,D_46_last_round2,B_2_last_round2,S_12_last_rou

In [7]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.8 MB/s eta 0:00:00:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [8]:
class DartEarlyStopping(object):

    def __init__(self, data_name, monitor_metric, stopping_round):
        self.data_name = data_name
        self.monitor_metric = monitor_metric
        self.stopping_round = stopping_round
        self.best_score = None
        self.best_model = None
        self.best_score_list = []
        self.best_iter = 0

    def _is_higher_score(self, metric_score, is_higher_better):
        if self.best_score is None:
            return True
        return (self.best_score < metric_score) if is_higher_better else (self.best_score > metric_score)

    def _deepcopy(self, x):
    
        return pickle.loads(pickle.dumps(x))

    def __call__(self, env):
        evals = env.evaluation_result_list
        for data, metric, score, is_higher_better in evals:
            if data != self.data_name or metric != self.monitor_metric:
                continue
            if not self._is_higher_score(score, is_higher_better):
                if env.iteration - self.best_iter > self.stopping_round:
            
                    eval_result_str = '\t'.join([lgb.callback._format_eval_result(x) for x in self.best_score_list])
                    lgb.basic._log_info(f"Early stopping, best iteration is:\n[{self.best_iter+1}]\t{eval_result_str}") 
                    lgb.basic._log_info(f"You can get best model by \"DartEarlyStopping.best_model\"")
                    raise lgb.callback.EarlyStopException(self.best_iter, self.best_score_list)
                return
        
            self.best_model = self._deepcopy(env.model)
            self.best_iter = env.iteration
            self.best_score_list = evals
            self.best_score = score
            return
        raise ValueError("monitoring metric not found")


In [ ]:
%%time
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import pickle
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

# ====================================================
# Configurations
# ====================================================
class CFG:
    seed = 41
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True



# ====================================================
# Train & Evaluate
# ====================================================
def train_and_evaluate(train):
    
    features = [col for col in train.columns if col not in ['customer_ID', 'target']]
    
    cat_features = cat_features = [
        "B_38",
        "D_114",
        "D_117",
        "D_120",
        "D_63",
        "D_64",
        "D_66",
    ]
    cat_features = [f"{cf}_last" for cf in cat_features]
    
    print(len(features))
    oof=[]
    VER = 1
    
    params = {
        'objective': 'cross_entropy_lambda',
        'metric': CFG.metric,
        'boosting': CFG.boosting_type,
        'seed': CFG.seed,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.2,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 2,
        'min_data_in_leaf': 40,
        }
    
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
        print(' ')
        print('-'*50)
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
        lgb_train = lgb.Dataset(x_train, y_train,categorical_feature = cat_features)
        lgb_valid = lgb.Dataset(x_val, y_val,categorical_feature = cat_features)
        
        des = DartEarlyStopping("valid_1", "amex_metric", 2000)
        
        model = lgb.train(
           params = params,
           train_set = lgb_train,
           num_boost_round = 12500,
           valid_sets = [lgb_train, lgb_valid],
           early_stopping_rounds = 2000,
           verbose_eval = 500,
           feval = lgb_amex_metric,
           callbacks=[des],
           )    
        
        model=lgb.Booster(model_file=f'Models/lgbm_fold{fold}.txt')
        
        model = des.best_model
        
        model.save_model(f'Models/lgbm_fold{fold}_seed{CFG.seed}.txt')
        
        #Save best model
       # joblib.dump(model, f'Models/lgbm_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}.pkl')
        
        # Predict validation
        preds = model.predict(x_val)
        # Compute fold metric
        score = amex_metric(y_val, preds)
        
        # SAVE OOF
        df = train.loc[val_ind, ['customer_ID','target'] ].copy()
        df['oof_pred_lgbm'] = preds
        oof.append( df )
        
        print(f'Our fold {fold} CV score is {score}')
        del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
        gc.collect()
    
    oof = pd.concat(oof,axis=0,ignore_index=True).set_index('customer_ID')
    
    val_score = amex_metric(oof.target.values, oof.oof_pred_lgbm.values)
    print(f"Overall Amex metric Score: {val_score}")
    
    del train
    gc.collect()
    
    # Create a dataframe to store out of folds predictions
    
    path_train = "s3://quri-sage-maker-bucket/train.parquet"
    oof_lgbm = pd.read_parquet(path_train, columns=['customer_ID']).drop_duplicates()
    oof_lgbm['customer_ID_hash'] = oof_lgbm['customer_ID'].str[-16:].apply(int, base=16).astype('int64')
    oof_lgbm = oof_lgbm.set_index('customer_ID_hash')
    oof_lgbm = oof_lgbm.merge(oof, left_index=True, right_index=True)
    oof_lgbm = oof_lgbm.sort_index().reset_index(drop=True)
    oof_lgbm.to_csv(f'oof_lgbm_v{VER}.csv',index=False)

seed_everything(CFG.seed)
train_and_evaluate(train)    

770
 
--------------------------------------------------
Training fold 0 with 770 features...
[LightGBM] [Info] [cross_entropy_lambda:Init]: (objective) labels passed interval [0, 1] check
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.029747 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 133051
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 768
[LightGBM] [Info] [cross_entropy_lambda:BoostFromScore]: havg = 0.258933 -> initscore = -1.218928
[LightGBM] [Info] Start training from score -1.218928
[500]	training's binary_logloss: 0.769104	training's amex_metric: 0.783184	valid_1's binary_logloss: 0.821098	valid_1's amex_metric: 0.76553
[1000]	training's binary_logloss: 0.846682	training's amex_metric: 0.796725	valid_1's binary_logloss: 0.928745	valid_1's amex_metric: 0.774083
[1500]	training's binary_logloss: 0.848296	training's amex_metric: 0.809517	val

In [ ]:
oof_lgbm = pd.read_csv('oof_lgbm_v1.csv')
oof_lgbm.head()